# Creating the OD Matrices
- Matrix 0: shortest trips between centroids
- Baseline: pop density and exp(normalized distance) -> gravity model baseline like Yap et al.
- Matrix set 1: equalizing for median income, education level, number of schools and number of jobs SEPARATELY
- Matrix set 2: equalizing for different attributes in O and D. O/D equalized for education level/number of schools, median income/number of jobs

In [1]:
import timeit
start = timeit.default_timer()
import sys
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
import osmnx as nx
import shapely
import multiprocess as mp
import multiprocessing
import numpy as np
import math
import igraph as ig
from ta_lab.assignment.assign import frank_wolfe
from ta_lab.assignment.line import *
from ta_lab.assignment.graph import *
from ta_lab.assignment.shortest_path import ShortestPath as SPP

crs_fr = 2154


## Custom functions

In [2]:
#--- Custom function (Anastassia)
# Create a dictionary of attributes (useful for networkX)
%run -i packages.py
def make_attr_dict(*args, **kwargs): 
    
    argCount = len(kwargs)
    
    if argCount > 0:
        attributes = {}
        for kwarg in kwargs:
            attributes[kwarg] = kwargs.get(kwarg, None)
        return attributes
    else:
        return None # (if no attributes are given)

In [3]:
#--- Custom function (Jin)
# equalize an OD for the same attribute in O and D
def equalization_all(od, variable, colname, delta, centroids): 
    
    od_ = od.copy()
    variable_ = variable.copy()
    
    variable_average = np.mean(variable_[colname]) 
    
    variable_['weight'] = variable_[colname].apply(lambda x: (x/variable_average)**-delta)

    i =0
    for val in variable_['ig']:
        weight = variable_.loc[variable_['ig']==val]['weight'].iloc[0]
        try:
            od_[centroids.index(val)] *= weight 
            od_.loc[centroids.index(val)] *= weight 
        except:
            continue
#             print(val, ' not found')
        i +=1
    
    return od_

In [4]:
#--- Custom function to use the function above in a batch
def clean_data_with_od_matrices(nodes_carbike_centroids_RER_complete, baseline_df, centroids, COLOFINTEREST, delta):
    col_tokeep = ['osmid', 'ig', 'CODE_IRIS', COLOFINTEREST]
    COLOFINTEREST_df = nodes_carbike_centroids_RER_complete.loc[nodes_carbike_centroids_RER_complete['centroid'] == True].copy()
    COLOFINTEREST_df = COLOFINTEREST_df[col_tokeep]
    
    OD_equalization = equalization_all(baseline_df, COLOFINTEREST_df, COLOFINTEREST, delta, centroids)
    
    OD_equalization_name = "OD_equalization_" + COLOFINTEREST + "_" + str(delta)
    
    return {OD_equalization_name: OD_equalization}

In [5]:
#--- Custom function (adapted from Jin)
# equalize an OD for DIFFFERENT attributes in O and D

def equalization_all_2attributes(od, variable, colnameO, colnameD, delta, centroids): 
    
    od_ = od.copy()
    variable_ = variable.copy()
    
    variable_average1 = np.mean(variable_[colnameO])
    variable_average2 = np.mean(variable_[colnameD])
    
    #here we keep -delta because we want to penalize the high values
    # e.g low income is prioritized 
    variable_['weightO'] = variable_[colnameO].apply(lambda x: (x / variable_average1) ** -delta)

    # here we use +delta because we want to penalize the low values
    # e.g high number of jobs in an area is prioritzed
    variable_['weightD'] = variable_[colnameD].apply(lambda x: (x / variable_average2) ** delta) #
    
    i = 0
    for val in variable_['ig']:
        weightO = variable_.loc[variable_['ig'] == val]['weightO'].iloc[0]
        weightD = variable_.loc[variable_['ig'] == val]['weightD'].iloc[0]
        try:
            od_.loc[centroids.index(val)] *= weightO #row = origin
            od_[centroids.index(val)] *= weightD #column = destination
        except:
            continue
        i += 1
    
    return od_


In [6]:
#--- Custom function to use the function above in a batch
def equalization_with_2attributes(nodes_carbike_centroids_RER_complete, baseline_df, centroids, COLOFINTEREST1, COLOFINTEREST2, delta):
    col_tokeep = ['osmid', 'ig', 'CODE_IRIS', COLOFINTEREST1, COLOFINTEREST2]
    COLSOFINTEREST_df = nodes_carbike_centroids_RER_complete.loc[nodes_carbike_centroids_RER_complete['centroid'] == True].copy()
    COLSOFINTEREST_df = COLSOFINTEREST_df[col_tokeep]
    
    equalized_od = equalization_all_2attributes(baseline_df, COLSOFINTEREST_df, COLOFINTEREST1, COLOFINTEREST2, delta, centroids)
    
    equalized_od_name = "OD_equalization_" + COLOFINTEREST1 + "_O_"+ COLOFINTEREST2 + "_D_delta_" + str(delta)
    
    return {equalized_od_name: equalized_od}


In [7]:
#--- Shapes

# GPM outline
GPM = gpd.read_file('data/raw/GPM.geojson').to_crs(crs_fr)

# IRIS codes and shapes 
IRIS_GPM = gpd.read_file('data/raw/IRIS_GPM.geojson')

## Creating the network in both NetworkX and igraph

In [8]:
#--- Create the network in NetworkX
# Retrieve edges
edges_with_id = pd.read_csv('data/clean/initial_network_edges_complete.csv')
edges_with_id["geometry"] = edges_with_id.apply(lambda x: shapely.wkt.loads(x.geometry), axis = 1)
edges_with_id = gpd.GeoDataFrame(edges_with_id, geometry = 'geometry', crs = 4326).to_crs(2154)

# Retrieve nodes
nodes_carbike_centroids_RER_complete = pd.read_csv('data/clean/initial_network_nodes_complete.csv')
nodes_carbike_centroids_RER_complete["geometry"] = nodes_carbike_centroids_RER_complete.apply(lambda x: shapely.wkt.loads(x.geometry), axis = 1)
nodes_carbike_centroids_RER_complete = gpd.GeoDataFrame(nodes_carbike_centroids_RER_complete, geometry = 'geometry', crs = 2154)

# Create the attr_dict
nodes_carbike_centroids_RER_complete["attr_dict"] = nodes_carbike_centroids_RER_complete.apply(lambda x: make_attr_dict(
                                                                  nodetype = x.nodetype,
                                                                  centroid = x.centroid,
                                                                  RER = x.RER,
                                                                  IRIS = x.CODE_IRIS,
                                                                  pop_dens = x.pop_density,
                                                                  active_pop_density = x.active_pop_density,
                                                                  school_pop_density = x.school_pop_density,
                                                                  num_schools = x.school_count,
                                                                  num_jobs = x.num_jobs,
                                                                  ),
                                                                  axis = 1) 

# Create Graph with all nodes and edges
G = nx.from_pandas_edgelist(edges_with_id, source='x', target='y', edge_attr=True)
G.add_nodes_from(nodes_carbike_centroids_RER_complete.loc[:,["osmid", "attr_dict"]].itertuples(index = False))

In [9]:
#--- Moving from NetworkX to igraph
g_igraph = ig.Graph()
networkx_graph = G
g_igraph = ig.Graph.from_networkx(networkx_graph)

# eids: "conversion table" for edge ids from igraph to nx 
eids_nx = [tuple(sorted(literal_eval(g_igraph.es(i)["edge_id"][0]))) for i in range(len(g_igraph.es))]
eids_ig = [i for i in range(len(g_igraph.es))]
eids_conv = pd.DataFrame({"nx": eids_nx, "ig": eids_ig})

# nids: "conversion table" for node ids from igraph to nx
nids_nx = [g_igraph.vs(i)["_nx_name"][0] for i in range(len(g_igraph.vs))]
nids_ig = [i for i in range(len(g_igraph.vs))]
nids_conv = pd.DataFrame({"nx": nids_nx, "ig": nids_ig})


In [10]:
nids_conv['nx'] = nids_conv['nx'].astype(int)

# combine the conversion table with nodes_carbike_centroids_RER_complete
nodes_carbike_centroids_RER_complete = nodes_carbike_centroids_RER_complete.merge(nids_conv, left_on = "osmid", right_on = "nx", how = "left")
nodes_carbike_centroids_RER_complete = nodes_carbike_centroids_RER_complete.drop(columns = ["nx"])

In [11]:
# Isolate centroids
from itertools import combinations
seq = g_igraph.vs.select(centroid_eq = True)
centroids = [v.index for v in seq]
centroids = centroids[0:2] #for testing purposes 
node_combinations = list(combinations(centroids, 2))

## Matrix 0: shortest path between each pair of centroids

In [12]:
%%time
# Create OD matrix
def process_node(args):
    start_node, end_node = args
    global g_igraph
    shortest_path_length = g_igraph.shortest_paths_dijkstra(source=start_node, target=end_node, weights='weight')[0][0]
    return (start_node, end_node, shortest_path_length)

if __name__ == '__main__':
    # Number of processes (cores) to use for parallel processing
    num_processes = mp.cpu_count()
    global g_igraph

    # Create a pool of processes
    pool = mp.Pool(processes=num_processes)

    # Apply the function to each node combination using parallel processing
    results = pool.map(process_node, node_combinations)

    # Create a dictionary to store the shortest path lengths
    output = {}
    for start_node, end_node, shortest_path_length in results:
        if start_node not in output:
            output[start_node] = {}
        output[start_node][end_node] = shortest_path_length

    # Create an empty adjacency matrix
    matrix = np.zeros((len(centroids), len(centroids)))

    # Fill the adjacency matrix with shortest path lengths
    for i, start_node in enumerate(centroids):
        for j, end_node in enumerate(centroids):
            if start_node in output and end_node in output[start_node]:
                matrix[i, j] = output[start_node][end_node]
                matrix[j, i] = output[start_node][end_node]

    # Close the pool
    pool.close()
    pool.join()

print(matrix.shape)


<timed exec>:5: DeprecationWarning: Graph.shortest_paths() is deprecated; use Graph.distances() instead


(2, 2)
CPU times: user 22.2 ms, sys: 41 ms, total: 63.2 ms
Wall time: 259 ms


## Baseline: population densities and exponential term with normalised distance

In [13]:
%%time

def process_node(args):
    global matrix
    o, d = args
    if o == d:
        return (o, d, 0)
    else:
        normalized_dist = matrix[o][d] / matrix.max()
        demand = (
            (g_igraph.vs[centroids[o]]['pop_dens'] * g_igraph.vs[centroids[d]]['pop_dens'])
            * dist_decay * np.exp(-1 * normalized_dist)
        )
        return (o, d, demand)

if __name__ == '__main__':
    baseline = np.zeros((len(centroids), len(centroids)))
    maxtrips = 100
    dist_decay = 1

    num_processes = mp.cpu_count()
    pool = mp.Pool(processes=num_processes)

    # Create node combinations
    node_combinations = [(o, d) for o in range(len(centroids)) for d in range(len(centroids))]

    # Calculate demand for each node combination using multiprocessing
    results = pool.map(process_node, node_combinations)

    # Update baseline matrix with calculated demand
    for o, d, demand in results:
        baseline[o][d] = demand

    # Normalize the matrix to the number of maxtrips
    baseline = ((baseline / baseline.max()) * maxtrips)

    # Round up to ensure each journey is made at least once
    baseline = np.ceil(baseline).astype(int)
    baseline_df = pd.DataFrame(baseline)

    pool.close()
    pool.join()


CPU times: user 17.8 ms, sys: 34.6 ms, total: 52.4 ms
Wall time: 60.1 ms


In [14]:
%%time
# Calculate demand between each origin and destination
# NO MULTIPROCESSING
baseline = np.zeros((len(centroids), len(centroids)))
maxtrips = 100
dist_decay = 1

for o in range(0, len(centroids)):
    for d in range(0, len(centroids)):
        if o == d:
            # do not insert demand down the spine - no trips where origin = destination
            baseline[o][d] = 0
        else:
            # normalize the current travel time versus the largest travel time between nodes in the matrix
            normalized_dist = matrix[o][d] / matrix.max()

            #  here, demand is a function of the product of the population of the origin and
            #  the destination - but reduced by the distance between them. 'Gravity demand'
            baseline[o][d] = ((g_igraph.vs[centroids[o]]['pop_dens'] * g_igraph.vs[centroids[d]]['pop_dens']) * dist_decay * np.exp(-1 * normalized_dist))

# we normalize the matrix to the number of maxtrips
baseline = ((baseline / baseline.max()) * maxtrips)

# we round up - to ensure each journey is made at least once
baseline = np.ceil(baseline).astype(int)
baseline_df = pd.DataFrame(baseline)

CPU times: user 276 µs, sys: 177 µs, total: 453 µs
Wall time: 450 µs


## Matrix Set 1: equalizing for median income, education level, number of schools and number of jobs SEPARATELY


In [15]:
%%time

def process_data(args):
    COLOFINTEREST = args
    delta_list = [0.5, 1, 1.5]
    results = {}
    for delta in delta_list:
        result = clean_data_with_od_matrices(nodes_carbike_centroids_RER_complete, baseline_df, centroids, COLOFINTEREST, delta)
        results.update(result)
    return results

if __name__ == '__main__':
    num_processes = mp.cpu_count()

    # Create a pool of processes
    pool = mp.Pool(processes=num_processes)    
    COLOFINTEREST_list = ['median_income', 'school_count', 'num_jobs']
    arguments = [COLOFINTEREST for COLOFINTEREST in COLOFINTEREST_list]
    results = pool.map(process_data, arguments)
    pool.close()
    pool.join()


CPU times: user 19.7 ms, sys: 36.2 ms, total: 56 ms
Wall time: 2.69 s


## Matrix Set 2: equalize for O/D attributes median income/ number of jobs, education level/number of schools

In [16]:
%%time
def process_combination(combination):
    COLOFINTEREST1, COLOFINTEREST2 = combination
    delta_list = [0.5, 1, 1.5]
    results = {}
    for delta in delta_list:
        result = equalization_with_2attributes(nodes_carbike_centroids_RER_complete, baseline_df, centroids, COLOFINTEREST1, COLOFINTEREST2, delta)
        results.update(result)
    return results

if __name__ == '__main__':
    combinations = [['median_income', 'num_jobs'], ['edu_level', 'school_count']]
    num_processes = mp.cpu_count()
    pool = mp.Pool(processes=num_processes)
    Results = pool.map(process_combination, combinations)
    pool.close()
    pool.join()


CPU times: user 18.4 ms, sys: 36.4 ms, total: 54.8 ms
Wall time: 5.64 s


In [17]:
stop = timeit.default_timer()

print('Time: ', stop - start)
(stop - start)/60

Time:  38.85781563299679


0.6476302605499465

## Assign traffic flow

In [58]:
#--- Create dataframe of edges compatible with frank_wolfe function

# goal columns: edge name, source, target, free flow time, capacity, alpha, beta

g_df = nx.to_pandas_edgelist(G)

# Create compatible edge names
g_df['edge'] = g_df.index + 1
g_df['edge'] = g_df['edge'].apply(lambda x: 'E'+ str(x).zfill(4))

g_df

# Adding the columns we don't have from the NetworkX network
g_df = g_df[['edge', 'source', 'target', 'length', 'geometry', 'id']]
g_df['capacity'] = 1e15
g_df['alpha'] = 0.15 #no idea how this is set
g_df['beta'] = 4.0 #same here

# Create compatible node names based on the osmIDs
g_df['source'] = g_df['source'].apply(lambda x: 'N'+ str(x).zfill(5))
g_df['target'] = g_df['target'].apply(lambda x: 'N'+ str(x).zfill(5))
g_df.reset_index(inplace=True)

# We have to explicitly say, and assume, that each link is a two-way road
g_df2 = g_df.copy()
g_df2['source'] = g_df['target']
g_df2['target'] = g_df['source']
g_df2['edge'] = g_df2.index + 1 + len(g_df)
g_df2['edge'] = g_df2['edge'].apply(lambda x: 'E'+ str(x).zfill(4))
g_df = pd.concat([g_df, g_df2])
geoms = g_df[['edge', 'geometry', 'index']]

# Clean-up
g_df.drop(['geometry', 'index'], axis=1, inplace=True)


# Save to csv
g_df.to_csv("data/clean/test_network.csv", index=False)


In [19]:
#--- Create network compatible with frank_wolfe function
nt = Network('net')
node = Vertex("a")

# Use the file created above
with open("data/clean/test_network.csv") as fo:
    lines = fo.readlines()[1:]
    for ln in lines:
        eg = ln.split(',')
        nt.add_edge(Edge(eg))


nt.init_cost()       

In [20]:
#--- Make it a batch run

# Gather all result OD matrices
results.extend(Results)
OD_matrix_names = []
OD_matrix = []

for result in results:
    OD_matrix_names.append(list(result.keys()))
    OD_matrix.append(list(result.values()))

OD_matrices_names = [item for sublist in OD_matrix_names for item in sublist]
OD_matrices = [dataframe for sublist in OD_matrix for dataframe in sublist]

# create dictionary of igraph ID to modified osmID
centroid_igraph_to_mod_osmID = {}
for i in range(len(centroids)):
    centroid_igraph_to_mod_osmID[i] = nodes_carbike_centroids_RER_complete.loc[nodes_carbike_centroids_RER_complete['ig'] == centroids[i]]['osmid'].apply(lambda x: 'N'+ (str(x) + '.0').zfill(5)).values[0]


# run frank-wolfe on all of them 
dicts = []
for name in OD_matrices_names:
    vol2 = None

    # Get OD matrix
    OD = OD_matrices[OD_matrices_names.index(name)]

    # Rename the columns and rows according to the modified osmID 
    OD = OD.rename(columns = {i : centroid_igraph_to_mod_osmID[i] for i in range(len(OD))}) #rename index of centroid as osmid of centroid
    OD.index = OD.columns

    # From all centroids to all centroids
    origins = OD.columns
    destinations = origins
    
    vol2 = frank_wolfe(nt, OD, origins, destinations)
    dicts.append(vol2)


## Compute benefit metric for all gaps (work in progress)

In [118]:
# WITH MULTIPROCESSING 

# Function to process each chunk
def process_chunk(chunk):
    chunk['path'] = chunk.path.apply(eval)
    chunk["B_star"] = chunk.apply(lambda x: 
                                    np.sum([dicts[0][g_df.loc[g_df['id'] == i]['edge'].values[0]] * \
                                            g_igraph.es[i]["length"] \
                                            for i in x.path]), 
                                    axis=1)
    chunk["B"] = chunk["B_star"] / chunk["length"]
    return chunk

# Read the CSV file in chunks using multiprocessing
if __name__ == '__main__':
    
    # Define the file path
    file_path = './data/clean/mygaps2.csv'

    # Define the chunk size
    chunk_size = 500
    num_processes = multiprocessing.cpu_count()
    pool = mp.Pool(processes=num_processes)

    chunks = []
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)

    results = pool.map(process_chunk, chunks)
    pool.close()
    pool.join()

# Concatenate the processed chunks into a single DataFrame
mygaps = pd.concat(results)

# Sort gaps by descending benefit metric
mygaps = mygaps.sort_values(by="B", ascending=False).reset_index(drop=True)

# Display the resulting DataFrame
print(mygaps)


KeyboardInterrupt: 

In [119]:
# multiprocessing part 2

# Function to process each chunk
def process_chunk(chunk):
    # Read the CSV file in each process
    chunk['path'] = chunk.path.apply(eval)
    chunk["B_star"] = chunk.apply(lambda x: 
                                    np.sum([dicts[0][g_df.loc[g_df['id'] == i]['edge'].values[0]] * \
                                            g_igraph.es[i]["length"] \
                                            for i in x.path]), 
                                    axis=1)
    chunk["B"] = chunk["B_star"] / chunk["length"]
    return chunk

if __name__ == '__main__':
    # Define the file path
    file_path = './data/clean/mygaps2.csv'

    # Define the chunk size
    chunk_size = 500
    num_processes = mp.cpu_count()
    pool = mp.Pool(processes=num_processes)

    results = pool.map(process_chunk, pd.read_csv(file_path, chunksize=chunk_size))
    pool.close()
    pool.join()

    # Concatenate the processed chunks into a single DataFrame
    mygaps = pd.concat(results)

    # Sort gaps by descending benefit metric
    mygaps = mygaps.sort_values(by="B", ascending=False).reset_index(drop=True)

    # Display the resulting DataFrame
    mygaps


In [114]:
# NOT MULTIPROCESSING
mygaps = pd.read_csv('./data/clean/mygaps2.csv', nrows=500)
mygaps['path'] = mygaps.path.apply(lambda x: eval(x))

mygaps["B_star"] = mygaps.apply(lambda x: 
                                        np.sum([dicts[0][g_df.loc[g_df['id'] == i]['edge'].values[0]] * \
                                                g_igraph.es[i]["length"] \
                                                for i in x.path]), 
                                        axis = 1)

mygaps["B"] = mygaps["B_star"] / mygaps["length"] # B(g) normed to length

# sort gaps by descending benefit metric
mygaps = mygaps.sort_values(by = "B", ascending = False).reset_index(drop = True)
mygaps


,path,length,path_nx,o_nx,d_nx,B_star,B
0,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",48.035217,"[(2660, 605558404), (605558404, 605558413), (6...",[,],1755.159241,36.539010
1,"[44691, 40872, 40871, 44690, 40706, 40694, 406...",189.035053,"[(2660, 605558404), (605558404, 605558413), (1...",[,],5104.326154,27.002009
2,"[44691, 40872, 40871, 44690, 40706, 40694, 406...",190.105948,"[(2660, 605558404), (605558404, 605558413), (1...",[,],5104.326154,26.849902
3,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",18.750942,"[(2660, 605558404), (605558404, 605558413), (6...",[,],0.000000,0.000000
4,"[44691, 40872, 40871, 44690, 40860, 40859, 408...",22.766010,"[(2660, 605558404), (605558404, 605558413), (1...",[,],0.000000,0.000000
...,...,...,...,...,...,...,...
495,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",37.974624,"[(2660, 605558404), (605558404, 605558413), (6...",[,],0.000000,0.000000
496,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",34.159859,"[(2660, 605558404), (605558404, 605558413), (6...",[,],0.000000,0.000000
497,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",33.783334,"[(2660, 605558404), (605558404, 605558413), (6...",[,],0.000000,0.000000
498,"[44691, 40872, 40867, 40863, 40862, 40791, 407...",35.468790,"[(2660, 605558404), (605558404, 605558413), (6...",[,],0.000000,0.000000


In [ ]:
# weekend / monday plan
# read EA stuff, do home stuff and gym over the weekend. NO THESIS!
# run the code overnight on Sunday night, see if the multiprocessing thin gets done by Monday morning (doesn't really matter anyway)
# do the todos below on Monday



# TODO after meeting Trivik
# maybe need to NOT normalize number of trips
# maybe need to have realistic capacity
# normalize the weights if you want but that's it
